In [24]:
import pandas as pd
from meteostat import Stations
import os

In [90]:
from meteostat import Hourly

Hourly.chunked = False

data = Hourly('KARB0')
data = data.fetch()

In [91]:
time_range = pd.date_range('2006-01-01 00:00:00', '2022-10-16 23:00:00', freq='H') ##creating time range
time_range=pd.DataFrame(time_range, columns=['time'])
time_range.set_index('time',inplace=True)
data=time_range.merge(data,how='left',left_index=True, right_index=True)

In [92]:
data['Timestamp'] = data.index
data['Year']       = data['Timestamp'].astype('M8[Y]')
data['DayOfYear']  = (data['Timestamp'] - data['Year']).astype('timedelta64[D]')
data['Hour']       = data['Timestamp'].dt.hour + 1
data['HourOfYear'] = data['DayOfYear'] * 24 + data['Hour']
data

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,Timestamp,Year,DayOfYear,Hour,HourOfYear
time,,,,,,,,,,,,,,,,
2006-01-01 00:00:00,1.0,-1.1,86.0,NaN,NaN,240.0,14.8,NaN,NaN,NaN,NaN,2006-01-01 00:00:00,2006-01-01,0.0,1,1.0
2006-01-01 01:00:00,1.1,-0.1,92.0,0.0,NaN,240.0,14.8,NaN,1009.0,NaN,NaN,2006-01-01 01:00:00,2006-01-01,0.0,2,2.0
2006-01-01 02:00:00,0.6,0.0,96.0,NaN,NaN,250.0,13.0,NaN,1009.5,NaN,NaN,2006-01-01 02:00:00,2006-01-01,0.0,3,3.0
2006-01-01 03:00:00,1.1,-0.1,92.0,NaN,NaN,250.0,13.0,NaN,1009.6,NaN,NaN,2006-01-01 03:00:00,2006-01-01,0.0,4,4.0
2006-01-01 04:00:00,1.1,-0.7,88.0,NaN,NaN,250.0,14.8,NaN,1010.5,NaN,NaN,2006-01-01 04:00:00,2006-01-01,0.0,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-16 19:00:00,13.3,3.4,51.0,0.0,NaN,270.0,22.0,NaN,1011.1,NaN,3.0,2022-10-16 19:00:00,2022-01-01,288.0,20,6932.0
2022-10-16 20:00:00,13.3,3.4,51.0,0.0,NaN,270.0,16.6,NaN,1010.5,NaN,3.0,2022-10-16 20:00:00,2022-01-01,288.0,21,6933.0
2022-10-16 21:00:00,12.2,2.9,53.0,0.0,NaN,280.0,18.4,NaN,1010.2,NaN,3.0,2022-10-16 21:00:00,2022-01-01,288.0,22,6934.0


In [94]:
dataresult = data.loc[:'2022-10-16 23:00:00'].copy(deep=True)
data=data.loc[:'2021-12-31 23:00:00']

In [115]:
all_time_avg_hr = data.groupby('HourOfYear',as_index=False).mean()
all_time_avg_hr.drop(columns=['snow', 'wpgt', 'tsun', 'coco', 'DayOfYear', 'Hour'],inplace=True)
all_time_avg_hr.columns = ['HourOfYear', 'ATAhr-temp', 'ATAhr-dwpt', 'ATAhr-rhum', 'ATAhr-prcp', 'ATAhr-wdir', 'ATAhr-wspd', 'ATAhr-pres']
all_time_avg_hr=all_time_avg_hr.fillna(0)


In [116]:
all_time_avg_day = data.groupby('DayOfYear',as_index=False).mean()
all_time_avg_day.drop(columns=['snow', 'wpgt', 'tsun', 'coco', 'HourOfYear', 'Hour'],inplace=True)
all_time_avg_day.columns = ['DayOfYear', 'ATAday-temp', 'ATAday-dwpt', 'ATAday-rhum', 'ATAday-prcp', 'ATAday-wdir', 'ATAday-wspd', 'ATAday-pres']
all_time_avg_day

,DayOfYear,ATAday-temp,ATAday-dwpt,ATAday-rhum,ATAday-prcp,ATAday-wdir,ATAday-wspd,ATAday-pres
0,0.0,-2.572776,-6.102156,77.657682,0.093980,223.717579,15.797838,1017.260112
1,1.0,-3.588281,-7.314062,76.359375,0.035621,212.907609,15.422977,1019.265354
2,2.0,-4.269271,-7.723698,77.653646,0.090407,218.204420,13.328646,1019.530366
3,3.0,-2.418033,-6.214481,75.767760,0.019672,233.950276,15.948634,1016.637110
4,4.0,-3.585901,-7.072063,77.459530,0.130744,242.946429,12.927415,1017.764499
...,...,...,...,...,...,...,...,...
361,361.0,-0.942708,-4.149740,79.393229,0.049363,212.647887,14.802887,1018.089737
362,362.0,-1.147917,-4.124740,81.005208,0.135015,210.262391,13.640885,1017.504545
363,363.0,-1.946615,-5.254948,78.968750,0.090741,208.764706,11.920052,1018.367553
364,364.0,-1.479427,-4.696875,79.611979,0.110616,218.038674,14.206771,1017.002394


In [123]:
data_last_5 = data[data['Year'].dt.year>2016]
last_5_avg_hr = data_last_5.groupby('HourOfYear',as_index=False).mean()
last_5_avg_hr.drop(columns=['snow', 'wpgt', 'tsun', 'coco', 'DayOfYear', 'Hour'],inplace=True)
last_5_avg_hr.columns = ['HourOfYear', 'L5Ahr-temp', 'L5Ahr-dwpt', 'L5Ahr-rhum', 'L5Ahr-prcp', 'ATAhr-wdir', 'L5Ahr-wspd', 'L5Ahr-pres']
#data_last_5[data_last_5['HourOfYear'==1]]
last_5_avg_hr=last_5_avg_hr.fillna(0)

In [124]:
last_5_avg_day = data_last_5.groupby('DayOfYear',as_index=False).mean()
last_5_avg_day.drop(columns=['snow', 'wpgt', 'tsun', 'coco', 'HourOfYear', 'Hour'],inplace=True)
last_5_avg_day.columns = ['DayOfYear', 'L5Aday-temp', 'L5Aday-dwpt', 'L5Aday-rhum', 'L5Aday-prcp', 'L5Aday-wdir', 'L5Aday-wspd', 'L5Aday-pres']
last_5_avg_day

,DayOfYear,L5Aday-temp,L5Aday-dwpt,L5Aday-rhum,L5Aday-prcp,L5Aday-wdir,L5Aday-wspd,L5Aday-pres
0,0.0,-4.346154,-7.376923,80.256410,0.054808,210.000000,12.152586,1020.086726
1,1.0,-2.516667,-5.115000,83.108333,0.034211,159.642857,10.974790,1019.786555
2,2.0,-0.926667,-3.220833,84.858333,0.180769,192.083333,12.591667,1013.560833
3,3.0,-2.177500,-5.631667,77.808333,0.003030,257.118644,16.235000,1012.970339
4,4.0,-5.280833,-9.196667,74.758333,0.011881,238.518519,13.081667,1015.699160
...,...,...,...,...,...,...,...,...
361,361.0,-2.428333,-5.344167,80.791667,0.051351,177.961165,10.241667,1019.515000
362,362.0,-0.855833,-3.367500,83.491667,0.281609,189.568966,13.109167,1016.966667
363,363.0,-2.246667,-5.638333,78.191667,0.260000,184.067797,13.551667,1016.977778
364,364.0,-2.519167,-5.389167,81.333333,0.246067,194.601770,13.383333,1014.996667


In [125]:
aa_historical = dataresult.merge(all_time_avg_hr,on='HourOfYear',how='left')
aa_historical = aa_historical.merge(last_5_avg_hr,on='HourOfYear',how='left')
aa_historical = aa_historical.merge(all_time_avg_day,on='DayOfYear',how='left')
aa_historical = aa_historical.merge(last_5_avg_day,on='DayOfYear',how='left')
aa_historical = aa_historical.set_index("Timestamp")

In [126]:
aa_historical.to_pickle('data/aa_historical.pkl')

In [127]:
aa_historical

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,...,ATAday-wdir,ATAday-wspd,ATAday-pres,L5Aday-temp,L5Aday-dwpt,L5Aday-rhum,L5Aday-prcp,L5Aday-wdir,L5Aday-wspd,L5Aday-pres
Timestamp,,,,,,,,,,,,,,,,,,,,,
2006-01-01 00:00:00,1.0,-1.1,86.0,NaN,NaN,240.0,14.8,NaN,NaN,NaN,...,223.717579,15.797838,1017.260112,-4.346154,-7.376923,80.256410,0.054808,210.0,12.152586,1020.086726
2006-01-01 01:00:00,1.1,-0.1,92.0,0.0,NaN,240.0,14.8,NaN,1009.0,NaN,...,223.717579,15.797838,1017.260112,-4.346154,-7.376923,80.256410,0.054808,210.0,12.152586,1020.086726
2006-01-01 02:00:00,0.6,0.0,96.0,NaN,NaN,250.0,13.0,NaN,1009.5,NaN,...,223.717579,15.797838,1017.260112,-4.346154,-7.376923,80.256410,0.054808,210.0,12.152586,1020.086726
2006-01-01 03:00:00,1.1,-0.1,92.0,NaN,NaN,250.0,13.0,NaN,1009.6,NaN,...,223.717579,15.797838,1017.260112,-4.346154,-7.376923,80.256410,0.054808,210.0,12.152586,1020.086726
2006-01-01 04:00:00,1.1,-0.7,88.0,NaN,NaN,250.0,14.8,NaN,1010.5,NaN,...,223.717579,15.797838,1017.260112,-4.346154,-7.376923,80.256410,0.054808,210.0,12.152586,1020.086726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-16 19:00:00,13.3,3.4,51.0,0.0,NaN,270.0,22.0,NaN,1011.1,NaN,...,223.482428,13.100263,1014.300000,9.480000,4.358333,72.808333,0.054639,262.0,15.611667,1013.324576
2022-10-16 20:00:00,13.3,3.4,51.0,0.0,NaN,270.0,16.6,NaN,1010.5,NaN,...,223.482428,13.100263,1014.300000,9.480000,4.358333,72.808333,0.054639,262.0,15.611667,1013.324576
2022-10-16 21:00:00,12.2,2.9,53.0,0.0,NaN,280.0,18.4,NaN,1010.2,NaN,...,223.482428,13.100263,1014.300000,9.480000,4.358333,72.808333,0.054639,262.0,15.611667,1013.324576


In [128]:
aa_historical.isna().sum()

temp              1014
dwpt              1231
rhum              1231
prcp             13410
snow            147192
wdir             24125
wspd              1416
wpgt            147192
pres              4880
tsun            147192
coco            140266
Year                 0
DayOfYear            0
Hour                 0
HourOfYear           0
ATAhr-temp           0
ATAhr-dwpt           0
ATAhr-rhum           0
ATAhr-prcp           0
ATAhr-wdir_x         0
ATAhr-wspd           0
ATAhr-pres           0
L5Ahr-temp           0
L5Ahr-dwpt           0
L5Ahr-rhum           0
L5Ahr-prcp           0
ATAhr-wdir_y         0
L5Ahr-wspd           0
L5Ahr-pres           0
ATAday-temp          0
ATAday-dwpt          0
ATAday-rhum          0
ATAday-prcp          0
ATAday-wdir          0
ATAday-wspd          0
ATAday-pres          0
L5Aday-temp          0
L5Aday-dwpt          0
L5Aday-rhum          0
L5Aday-prcp          0
L5Aday-wdir          0
L5Aday-wspd          0
L5Aday-pres          0
dtype: int6